## Qwen 3 being Game Changer

In [3]:
import os
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

# --- 1. Simple Python Function for Mode Detection ---
# (This function remains the same, as it determines *our* desired mode based on input)
def determine_thinking_mode_text(user_input: str) -> str:
    """
    Determines the AI thinking mode ('direct' or 'analytical')
    based on text length and keyword presence.

    Args:
        user_input: The user's text input string.

    Returns:
        'direct' or 'analytical'.
    """
    # Define keywords that typically indicate a need for deeper analysis
    complex_keywords = [
        'analyze', 'compare', 'explain the process',
        'synthesize', 'evaluate', 'troubleshoot',
        'implications', 'how it works', 'why',
        'describe the steps', 'discuss the impact',
        'break down', 'outline the key factors',
        'in detail', 'walk me through' # Added a few more variations
    ]

    # Define a length threshold (e.g., in characters). Adjust as needed.
    length_threshold_chars = 150 # Example: Treat inputs > 150 characters as potentially needing analysis

    input_lower = user_input.lower()

    # Check for presence of any complex keyword
    contains_complex_keyword = any(keyword in input_lower for keyword in complex_keywords)

    # Check if input exceeds the length threshold
    exceeds_length = len(user_input) > length_threshold_chars

    # Decision Logic: Go to analytical if keywords are present OR it's long
    if contains_complex_keyword or exceeds_length:
        print(f"--- Input classified as: ANALYTICAL ---")
        return 'analytical'
    else:
        print(f"--- Input classified as: DIRECT ---")
        return 'direct'

# --- 2. LangChain Setup and Prompt Templates ---

# Initialize the ChatOllama model
llm = None
try:
    # Use the ChatOllama class from the dedicated langchain_ollama package
    # We don't set temperature=0 by default here to potentially see more varied analytical responses
    llm = ChatOllama(model="qwen3:8b")
    print("--- Successfully initialized ChatOllama with Qwen 3:8b ---")
except Exception as e:
    print(f"Error initializing ChatOllama: {e}")
    print("Please ensure Ollama is installed, running, and 'qwen3:8b' is pulled.")
    print("Skipping LLM interaction.")

# Define a single base prompt template
# We will append the mode tag (/think or /no_think) to the user input in the code
base_prompt_template = ChatPromptTemplate.from_messages(
    [
        # Keep a general system prompt, the /think /no_think tags are the primary mode control for Qwen
        ("system", "You are a helpful AI assistant."),
        ("user", "{input_with_mode_tag}") # Placeholder for user input + mode tag
    ]
)


# --- 3. Example Inputs (Simulated Dataset) ---
example_inputs = [
    "What is the largest ocean?", # Should be Direct -> /no_think
    "Define machine learning.", # Should be Direct -> /no_think
    "Explain the process of photosynthesis step-by-step, detailing the light-dependent and light-independent reactions.", # Should be Analytical -> /think
    "Compare and contrast the political systems of parliamentary and presidential democracies, outlining their key differences and similarities.", # Should be Analytical -> /think
    "List 3 famous landmarks in Rome.", # Should be Direct -> /no_think
    "Analyze the potential long-term implications of climate change on global food security and discuss mitigation strategies.", # Should be Analytical -> /think
    "What is the formula for water?", # Should be Direct -> /no_think
    "Describe the steps involved in training a convolutional neural network for image classification, from data preparation to evaluation.", # Should be Analytical -> /think
    "Simply provide the capital of Japan.", # Should be Direct -> /no_think
    "Discuss the impact of the internet on social interactions and community building over the past two decades, breaking down the key changes.", # Should be Analytical -> /think
    # Add an example that is short but uses a complex keyword
    "Analyze this short sentence.", # Should be Analytical -> /think (keyword triggers)
    # Add an example that is long but potentially simple factual (may still go analytical due to length)
    "The history of the internet is a fascinating journey from a research project to a global communication network. It involved many key figures and technological advancements over several decades.", # Should be Analytical -> /think (length triggers)
]


# --- 4. Main Execution Logic with User Choice ---

if llm: # Only proceed if LLM was successfully initialized
    while True:
        print("\n" + "="*40)
        print(" AI Thinking Mode Demo (Text) ")
        print("="*40)
        print("Choose Input Method:")
        print("1. Use an example input")
        print("2. Enter custom input")
        print("3. Exit")
        print("="*40)

        choice = input("Enter your choice (1, 2, or 3): ")

        if choice == '3':
            print("Exiting demo.")
            break
        elif choice == '1':
            print("\n--- Select Example Input ---")
            # Display examples with predicted mode for user clarity
            print("Examples (predicted mode):")
            for i, example in enumerate(example_inputs):
                 predicted_mode = determine_thinking_mode_text(example) # Predict mode to show user
                 print(f"{i + 1}. [{predicted_mode.upper()}] {example}")

            while True:
                try:
                    example_index = int(input(f"Enter the number of the example (1-{len(example_inputs)}): ")) - 1
                    if 0 <= example_index < len(example_inputs):
                        user_input = example_inputs[example_index]
                        break # Valid input, exit inner loop
                    else:
                        print("Invalid number. Please try again.")
                except ValueError:
                    print("Invalid input. Please enter a number.")

        elif choice == '2':
            print("\n--- Enter Custom Input ---")
            user_input = input("Enter your text: ")
            if not user_input.strip(): # Use strip() to check for empty or whitespace-only input
                print("Input cannot be empty. Please try again.")
                continue # Skip to the next iteration of the main loop

        else:
            print("Invalid choice. Please enter 1, 2, or 3.")
            continue # Skip to the next iteration of the main loop

        # --- Process the chosen/entered input ---
        print(f"\nUser Input: {user_input}")

        # Determine the thinking mode using our simple function
        mode = determine_thinking_mode_text(user_input)

        # Append the appropriate mode tag to the user input for Qwen
        if mode == 'direct':
            input_with_mode_tag = f"{user_input} /no_think"
            ollama_mode_tag_used = "/no_think"
        else: # mode == 'analytical'
            input_with_mode_tag = f"{user_input} /think"
            ollama_mode_tag_used = "/think"

        # Create the LangChain chain: Base Prompt Template -> LLM
        # The base prompt template now just structures the message roles
        chain = base_prompt_template | llm

        # Invoke the chain and get the response from Qwen
        try:
            print(f"Sending to Qwen using OLLAMA mode tag: '{ollama_mode_tag_used}'...")
            # We pass the user input WITH the mode tag to the prompt
            response = chain.invoke({"input_with_mode_tag": input_with_mode_tag})
            print(f"\n--- AI Response (Prompted with '{ollama_mode_tag_used}') ---")
            print(response.content) # Qwen's raw output, including <think> if used

        except Exception as e:
             print(f"Error getting response from Qwen for input '{user_input}': {e}")
             print("Please ensure Ollama is serving the model correctly and check Ollama logs.")
             print("Sometimes restarting Ollama or pulling the model again can help.")


        print("\n" + "="*40) # Separator for next interaction

else:
    print("\nLLM interaction is not possible because initialization failed.")

--- Successfully initialized ChatOllama with Qwen 3:8b ---

 AI Thinking Mode Demo (Text) 
Choose Input Method:
1. Use an example input
2. Enter custom input
3. Exit

--- Enter Custom Input ---

User Input: How to plants grow without sunlight in forest ?
--- Input classified as: DIRECT ---
Sending to Qwen using OLLAMA mode tag: '/no_think'...

--- AI Response (Prompted with '/no_think') ---
<think>

</think>

Plants in forests grow without direct sunlight because they rely on **indirect light** and **canopy layering**. Here's how they manage to thrive in low-light conditions:

---

### 🌲 **1. Canopy Layers and Light Penetration**
Forests have **multiple layers** of vegetation:
- **Canopy layer**: Tall trees (like oaks, maples, or pines) form the top layer, blocking most direct sunlight.
- **Understory layer**: Shrubs, ferns, and smaller trees grow beneath the canopy, receiving **diffuse light** that filters through the leaves.
- **Forest floor**: Mosses, lichens, and shade-tolerant pla